# Combining and analysing the simulations done for testing restarts on SPICE
Using the lowdensity S probe at 10$^{\circ}$

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
# import flopter.spice.homogenise as hmg
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c

In [2]:
importlib.reload(spl)
importlib.reload(td)
# importlib.reload(hmg)
importlib.reload(fd)
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

In [3]:
def plot_raw_data_splopter(path_to_splopter, axes=None, homogenise_fl=False):
    label = str(path_to_splopter).split('/')[-1]
    print(label)
    
    splopter = spl.Splopter(path_to_splopter, reduce=desired_variables)
    splopter.prepare(denormaliser_fl=False, homogenise_fl=homogenise_fl, find_se_temp_fl=False)

    # Extract relevant arrays from the matlab file
    probe_indices = splopter.parser.get_probe_obj_indices()
    probe_current_e = 0.0
    probe_current_i = 0.0

    time = np.squeeze(splopter.tdata.t)[:-1]
    for index in probe_indices:
        probe_current_e += np.squeeze(splopter.tdata.objectscurrente)[index]
        probe_current_i += np.squeeze(splopter.tdata.objectscurrenti)[index]
    probe_bias = np.squeeze(splopter.tdata.diagnostics['ProbePot'])
    probe_current_tot = probe_current_i + probe_current_e
    
    print(splopter.tdata.diagnostics)
    
    if axes is None:
        fig, axes = plt.subplots(2)
    
    axes[0].set_title(label)
    axes[0].plot(probe_bias, label=label)
#     axes[0].plot(universal_probe_bias)
    axes[0].legend()
    axes[1].plot(probe_current_tot, label=label)
#     axes[1].plot(time, probe_current_i)
#     axes[1].plot(time, probe_current_e)
    axes[1].legend()
    
    return splopter
   

In [4]:
def plot_concatted_current(splopters, trim_concat=False, ax=None):
    label = str(splopters[0].data_dir).split('/')[-2].split('.')[0]
    if label[-1] == 'c':
        trim_concat = True
    concatted_time = np.array([])
    concatted_current = np.array([])
    if trim_concat:
        lengths = []
        
    if ax is None:
        fig, ax = plt.subplots()
    
    for s in splopters:
        # Extract relevant arrays from the matlab file
        probe_indices = s.parser.get_probe_obj_indices()
        probe_current_e = 0.0
        probe_current_i = 0.0

        time = np.squeeze(s.tdata.t)[:-1]
        if len(concatted_time) > 0:
            offset = concatted_time[-1]
        else:
            offset = 0
        
        for index in probe_indices:
            probe_current_e += np.squeeze(s.tdata.objectscurrente)[index]
            probe_current_i += np.squeeze(s.tdata.objectscurrenti)[index]
        probe_bias = np.squeeze(s.tdata.diagnostics['ProbePot'])
        probe_current_tot = probe_current_i + probe_current_e
        
        if trim_concat:
            indices_oi = np.where((probe_current_tot != 0.0) & (probe_current_tot <= 100.0) & (probe_current_tot >= -500.0))
            
            concatted_current = np.concatenate([concatted_current, probe_current_tot[indices_oi]])
            concatted_time = np.concatenate([concatted_time, offset + np.arange(len(time[indices_oi]))])
            
            ax.axvline(x=offset, **c.AX_LINE_DEFAULTS)
            lengths.append(offset)
        else:
            print(f'time: {len(time)}, current: {len(probe_current_tot)}')
            concatted_current = np.concatenate([concatted_current, probe_current_tot])
            concatted_time = np.concatenate([concatted_time, offset + time])
    
    if trim_concat:
        ax.plot(concatted_time, concatted_current, label=label)
        return concatted_time, concatted_current
    else:
        ax.plot(concatted_time, concatted_current, label=label)
        return concatted_time, concatted_current

In [5]:
def plot_concatted_bias(splopters, trim_concat=False, ax=None):
    label = str(splopters[0].data_dir).split('/')[-2].split('.')[0]
    
    concatted_time = np.array([])
    concatted_voltage = np.array([])
    if trim_concat:
        lengths = []
        
    if ax is None:
        fig, ax = plt.subplots()
    
    for s in splopters:
        probe_bias = np.squeeze(s.tdata.diagnostics['ProbePot'])
#         print(probe_bias.shape)
        if trim_concat:
            concatvolt_len = len(concatted_voltage)
            concatted_voltage = np.concatenate([concatted_voltage, probe_bias[concatvolt_len:]])
            ax.axvline(x=concatvolt_len, **c.AX_LINE_DEFAULTS)
            lengths.append(concatvolt_len)
        elif isinstance(probe_bias, np.ndarray) and len(probe_bias.shape) > 0:
            concatted_voltage = np.concatenate([concatted_voltage, probe_bias])
    
    ax.plot(concatted_voltage, label=label)
    return concatted_voltage

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [6]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

In [7]:
list(spice_dir.glob('marconi/*/restart_test/*'))

[PosixPath('/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_20hr_multirestart_recomp_4c'),
 PosixPath('/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_10hr_multirestart_2r'),
 PosixPath('/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_10hr_multirestart_r'),
 PosixPath('/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_1hr'),
 PosixPath('/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_1hr.1'),
 PosixPath('/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_1hr_2.14'),
 PosixPath('/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_1hr_atrestart'),
 PosixPath('/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_4c'),
 PosixPath('/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_hdf5_4c'),
 PosixPath('/home/jleland/data/external_big/s

In [8]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [9]:
skippable_scans = {
    'marconi/spice2/lowdens_anglescan/inpartest_as',
    'marconi/spice2/lowdens_anglescan/angled_tiltscan_ext',
    'marconi/spice2/restart_test/sprobe_9hr_multirestart',
    'marconi/spice2/restart_test/sprobe_1hr',
    'marconi/spice2/restart_test/sprobe_1hr.1',
    'marconi/spice2/restart_test/sprobe_1hr_atrestart',
    'marconi/spice2/restart_test/sprobe_4hr_multirestart_r',
    'marconi/spice2/restart_test/sprobe_4hr_multirestart_2r',
    'marconi/spice2/restart_test/sprobe_4hr_multirestart_c',
    'marconi/spice2/restart_test/sprobe_4hr_multirestart_2c',
    'marconi/spice2/restart_test/sprobe_10hr_multirestart_r',
    'marconi/spice2/restart_test/sprobe_10hr_multirestart_2r',
    'marconi/spice2/restart_test/sprobe_control_mkomm',
    'marconi/spice2/restart_test/sprobe_control_2.13_hdf5',
    'marconi/spice2/restart_test/sprobe_20hr_multirestart_2.13_4c',
    'marconi/spice2/restart_test/sprobe_20hr_multirestart_2.13_hdf5_4c',
    'marconi/spice2/restart_test/sprobe_control',
    'marconi/spice2/restart_test/sprobe_control_recomp',
    'marconi/spice2/restart_test/sprobe_4hr_multirestart_c',
    'marconi/spice2/restart_test/sprobe_4hr_multirestart_2c',
    'marconi/spice2/restart_test/sprobe_20hr_multirestart_mkomm_4c',
    'marconi/spice2/restart_test/sprobe_control_mkomm_rel',
    'marconi/spice2/restart_test/sprobe_20hr_multirestart_4c',
    'marconi/spice2/restart_test/sprobe_20hr_multirestart_4c.1',
    'marconi/spice2/restart_test/sprobe_20hr_multirestart_4c.2',
    'marconi/spice2/restart_test/sprobe_20hr_multirestart_4r',
    'marconi/spice2/restart_test/sprobe_20hr_multirestart_recomp_4c',
    'marconi/spice2/restart_test/sprobe_control_2.13',
    'marconi/spice2/restart_test/sprobe_control_2-13-1_hdf5_',
    'marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_hdf5_4c',
    'marconi/spice2/restart_test/sprobe_control_2.13_',
#     'marconi/spice2/restart_test/sprobe_control_2-13-1_'
}
single_sims = {
    'marconi/spice2/restart_test/sprobe_control_2.13_',
    'marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_4c',
    'marconi/spice2/restart_test/sprobe_control_2-13-1_',
    'marconi/spice2/restart_test/sprobe_20hr_multirestart_2.14_4c',
    'marconi/spice2/restart_test/sprobe_1hr_2.14'
}

In [10]:
scans_searchstr = 'marconi/*/restart_test/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}

desired_variables = td.DEFAULT_REDUCED_DATASET | non_standard_variables

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        print(f'Single sim {scan} found')
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)
    
    print(f'{scan}: {all_run_dirs[scan]}\n')


marconi/spice2/restart_test/sprobe_control_2.13_redo: ['marconi/spice2/restart_test/sprobe_control_2.13_redo/backup_20191031-1811', 'marconi/spice2/restart_test/sprobe_control_2.13_redo/jobs.txt', 'marconi/spice2/restart_test/sprobe_control_2.13_redo/log.err', 'marconi/spice2/restart_test/sprobe_control_2.13_redo/log.ongoing.err', 'marconi/spice2/restart_test/sprobe_control_2.13_redo/log.ongoing.out', 'marconi/spice2/restart_test/sprobe_control_2.13_redo/log.out', 'marconi/spice2/restart_test/sprobe_control_2.13_redo/sprobe_control_2.13_redo.mat', 'marconi/spice2/restart_test/sprobe_control_2.13_redo/t-sprobe_control_2.13_redo.mat', 'marconi/spice2/restart_test/sprobe_control_2.13_redo/t-sprobe_control_2.13_redo00.mat', 'marconi/spice2/restart_test/sprobe_control_2.13_redo/t-sprobe_control_2.13_redo01.mat', 'marconi/spice2/restart_test/sprobe_control_2.13_redo/t-sprobe_control_2.13_redo03.mat', 'marconi/spice2/restart_test/sprobe_control_2.13_redo/t-sprobe_control_2.13_redo04.mat', 'ma

In [11]:
skippable = {}
scans = list(scans)
scans.sort()

In [12]:
for i, scan in enumerate(scans):
    print(i, scan)

0 marconi/spice2/restart_test/sprobe_1hr_2.14
1 marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_4c
2 marconi/spice2/restart_test/sprobe_20hr_multirestart_2.14_4c
3 marconi/spice2/restart_test/sprobe_control_2-13-1_
4 marconi/spice2/restart_test/sprobe_control_2.13_altsolv
5 marconi/spice2/restart_test/sprobe_control_2.13_redo


In [13]:
chosen_spl_path = spice_dir / all_run_dirs[scans[0]][0]
print(chosen_spl_path)

/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_1hr_2.14


## Check the 1hr Version 
To see if the probe bias is being set correctly.

In [14]:
import scipy.io as sio
tdata = sio.loadmat(chosen_spl_path / 't-sprobe_1hr_2.14.mat')

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "Esct" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "dPHIqn" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "pchi" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages

In [15]:
tdata.keys()

dict_keys(['__header__', '__version__', '__globals__', 'version', 'Nz', 'Nzmax', 'Ny', 'count', 'hpos', 'deltah', 'Npc', 'dt', 'dz', 'nproc', 'q', 'm', 'Temp', 'zg', 'yg', 'rho', 'Escz', 'Escy', 'surfacematrix', 'Pot', 'Potvac', 'sliceproc', 'orientation', 'edgecharge', 't', 'snumber', 'totalenergy', 'Esct', 'dPHIqn', 'pchi', 'bx', 'by', 'bz', 'npartproc', 'nodiagreg', 'diaghistories', 'fvarrays', 'fvbin', 'fvperparraycount', 'fvlimits', 'histlimits', 'timehistory', 'QnPot', 'ProbePot', 'iHistTopx1', 'iHistTopx2', 'iHistTopx3', 'eHistTopx1', 'eHistTopx2', 'eHistTopx3', 'iHistSEx1', 'iHistSEx2', 'iHistSEx3', 'eHistSEx1', 'eHistSEx2', 'eHistSEx3', 'flagm', 'equipotm', 'flag', 'itertime', 'injrate', 'objects', 'edges', 'diagm', 'objectsenum', 'objectscurrenti', 'objectscurrente', 'objectspowerfluxi', 'objectspowerfluxe', 'rho01', 'solw01', 'solns01', 'impactdiag01', 'emmitdiag01', 'rho02', 'solw02', 'solns02', 'impactdiag02', 'emmitdiag02', 'ksi', 'tau', 'mu', 'mksn0', 'mksTe', 'mksB', 'm

In [16]:
tdata['t']

array([[1.08676882e-02],
       [2.17353763e-02],
       [3.26030645e-02],
       ...,
       [1.52691019e+01],
       [1.52799695e+01],
       [1.52908372e+01]])

In [17]:
fig, ax = plt.subplots(2)
hand = ax[0].contourf(tdata['Pot'])
fig.colorbar(hand)
hand = ax[1].contourf(tdata['objectsenum'])
fig.colorbar(hand)

In [56]:
fig, ax = plt.subplots(2)
ax[0].plot(tdata['ProbePot'])
ax[1].plot(tdata['objectscurrente'][3])
ax[1].plot(tdata['objectscurrenti'][3])

Looks like the probe potential is still -10V but doesn't register on the diagnostic

## Check the 2.14 Full Sim 
With 4 hour full restarts (using -c)

In [18]:
for i, scan in enumerate(scans):
    print(i, scan)

0 marconi/spice2/restart_test/sprobe_1hr_2.14
1 marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_4c
2 marconi/spice2/restart_test/sprobe_20hr_multirestart_2.14_4c
3 marconi/spice2/restart_test/sprobe_control_2-13-1_
4 marconi/spice2/restart_test/sprobe_control_2.13_altsolv
5 marconi/spice2/restart_test/sprobe_control_2.13_redo


In [19]:
new_spl_path = spice_dir / all_run_dirs[scans[2]][0]
print(new_spl_path)

splopter_new = spl.Splopter(new_spl_path, reduce=desired_variables)
splopter_new.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False, backup_concat_fl=False)

/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_20hr_multirestart_2.14_4c
No a-file given, continuing without


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [20]:
splopter_new.tdata.t_dict['t']

array([[1.73883010e-01],
       [3.47766021e-01],
       [5.21649031e-01],
       ...,
       [3.47418255e+02],
       [3.47592138e+02],
       [3.47766021e+02]])

In [21]:
old_spl_path = spice_dir / all_run_dirs[scans[1]][0]
print(old_spl_path)

splopter_old = spl.Splopter(old_spl_path, reduce=desired_variables)
splopter_old.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)

/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_4c
Looking for a suitable backup to use instead.
Useable backup found at /home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191105-2109/t-sprobe_20hr_multirestart_2-13-1_4c.mat
No a-file given, continuing without
Loading backup backup_20191105-2109 (1 of 5) for current and bias concatenation
Loading backup backup_20191106-0119 (2 of 5) for current and bias concatenation
Stripping head for backup 2 of 5
Loading backup backup_20191106-0159 (3 of 5) for current and bias concatenation
Detected finish, stripping tail for backup 3 of 5
Stripping head for backup 3 of 5
Found an odd number of spike transitions


In [60]:
ctrl_spl_path = spice_dir / all_run_dirs[scans[3]][0]
print(ctrl_spl_path)

splopter_ctrl = spl.Splopter(ctrl_spl_path, reduce=desired_variables)
splopter_ctrl.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False, backup_concat_fl=False)

/home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_control_2-13-1_
No a-file given, continuing without


In [22]:
def get_backup_splopters(spl_path):
    backups = list(spl_path.glob('backup*'))
    backups.sort()

    bu_splopters = {}
    for i, bu_folder in enumerate(backups):
        print(f'{i}) {str(bu_folder)}')

        splopter = spl.Splopter(bu_folder, reduce=desired_variables)
        splopter.prepare(denormaliser_fl=False, homogenise_fl=False, find_se_temp_fl=False)
        bu_splopters[bu_folder] = splopter
    return bu_splopters

In [62]:
old_bu_splopters = get_backup_splopters(old_spl_path)
new_bu_splopters = get_backup_splopters(new_spl_path)

0) /home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191105-2109
No a-file given, continuing without
1) /home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191106-0119
No a-file given, continuing without
2) /home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191106-0159
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
3) /home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191106-0213
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
4) /home/jleland/data/external_big/spice/marconi/spice2/restart_test/sprobe_20hr_multirestart_2-13-1_4c/backup_20191106-0228
There are no backups to utilise. Carrying on, but expect erroneous results.
No 

In [63]:
for bu_folder in new_bu_splopters.keys():
    plot_raw_data_splopter(bu_folder)

plot_concatted_current(list(new_bu_splopters.values()), trim_concat=True)

backup_20200507-2059
No a-file given, continuing without
{'ProbePot': array([[ 0.  ],
       [ 0.  ],
       [ 0.  ],
       ...,
       [-5.85],
       [-5.85],
       [-5.85]])}
backup_20200509-0917
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
{'ProbePot': array([[0.  ],
       [0.  ],
       [0.  ],
       ...,
       [5.55],
       [5.55],
       [5.55]])}
backup_20200510-0413
No a-file given, continuing without
{'ProbePot': array([[ 0.  ],
       [ 0.  ],
       [ 0.  ],
       ...,
       [10.05],
       [10.05],
       [ 0.  ]])}


IndexError: index 1045 is out of bounds for axis 0 with size 1044

/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [64]:
# plot_concatted_current(list(new_bu_splopters.values()), trim_concat=True)
plot_raw_data_splopter(new_spl_path)

sprobe_20hr_multirestart_2.14_4c
No a-file given, continuing without
{'ProbePot': array([[ 0.  ],
       [ 0.  ],
       [ 0.  ],
       ...,
       [10.05],
       [10.05],
       [ 0.  ]])}


In [66]:
fig, ax = plt.subplots(2, sharey=True)

ax[0].plot(splopter_old.raw_data['I'])
ax[1].plot(splopter_new.raw_data['I'])

In [25]:
fig, ax = plt.subplots(2, 2, sharex='col')

ax[0][0].plot(splopter_old.raw_data['I'][:1100])
ax[1][0].plot(splopter_old.raw_data['V'][900:])
ax[0][1].plot(splopter_new.raw_data['I'])
ax[1][1].plot(splopter_new.raw_data['V'])

In [24]:
print(len(splopter_old.raw_data['I']))
print(len(splopter_old.raw_data['V']))
print(len(splopter_old.tdata.t))

print(len(splopter_new.raw_data['I']))
print(len(splopter_new.tdata.t_dict['ProbePot']))
print(len(splopter_new.raw_data['V']))
print(len(splopter_new.tdata.t))

1706
1707
1194
568524
568525
852786
2000


In [26]:
print(f'ta: \t {splopter_new.parser["geom"]["ta"]}')
print(f'tp: \t {splopter_new.parser["geom"]["tp"]}')
print(f'tc: \t {splopter_new.parser["geom"]["tc"]} \n')

ta: 	 2.0
tp: 	 3.0
tc: 	 1.0 



In [73]:
splopter_new.get_tdata_raw_iv(splopter_new.tdata)

(array([1.73883010e-01, 3.47766021e-01, 5.21649031e-01, ...,
        3.47244372e+02, 3.47418255e+02, 3.47592138e+02]),
 array([ 0.  ,  0.  ,  0.  , ..., 10.05, 10.05,  0.  ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]))

In [75]:
fig, ax = plt.subplots(2)

sweep_slc = slice(189509,568524)
index = np.arange(0,568525)
pot = np.squeeze(splopter_new.tdata.t_dict['ProbePot'])

print(f'index.shape = {index.shape}')
print(f'pot.shape = {pot.shape}')
print(f'current.shape = {splopter_new.raw_data["I"].shape}')

ax[0].plot(index[:-1], splopter_new.raw_data['I'])
ax[0].plot(index[sweep_slc], splopter_new.raw_data['I'][sweep_slc], color='tab:green')

pot_ax = ax[0].twinx()
pot_ax.plot(index, pot, color='tab:orange')
pot_ax.plot(index[sweep_slc], pot[sweep_slc], color='tab:red')


ax[1].plot(np.squeeze(splopter_new.tdata.t_dict['ProbePot'])[sweep_slc], splopter_new.raw_data['I'][sweep_slc])

index.shape = (568525,)
pot.shape = (568525,)
current.shape = (568524,)


In [77]:
df = pd.DataFrame({'voltage':pot[sweep_slc], 'current':splopter_new.raw_data['I'][sweep_slc]})
df

,voltage,current
0,-9.95,33.0
1,-9.95,36.0
2,-9.95,27.0
3,-9.95,35.0
4,-9.95,30.0
...,...,...
379010,10.05,0.0
379011,10.05,0.0
379012,10.05,0.0
379013,10.05,0.0


In [78]:
voltage_groups = df.groupby('voltage').groups

In [79]:
for voltage_group in voltage_groups:
    voltage_group_size = len(voltage_groups[voltage_group])
    if voltage_group_size > 5:
        print(voltage_group, voltage_group_size)

-9.950000000000001 947
-9.9 947
-9.85 947
-9.799999999999999 947
-9.75 947
-9.700000000000001 947
-9.65 947
-9.6 947
-9.549999999999999 947
-9.5 947
-9.450000000000001 947
-9.4 947
-9.35 947
-9.299999999999999 947
-9.25 947
-9.200000000000001 947
-9.15 947
-9.1 947
-9.049999999999999 947
-9.0 947
-8.950000000000001 947
-8.9 947
-8.85 947
-8.799999999999999 947
-8.75 947
-8.700000000000001 947
-8.65 947
-8.6 947
-8.549999999999999 947
-8.5 947
-8.450000000000001 947
-8.4 947
-8.35 947
-8.299999999999999 947
-8.25 947
-8.200000000000001 947
-8.15 947
-8.1 947
-8.049999999999999 947
-8.0 947
-7.950000000000001 947
-7.8999999999999995 947
-7.8500000000000005 947
-7.799999999999999 947
-7.75 947
-7.700000000000001 947
-7.6499999999999995 947
-7.6000000000000005 947
-7.549999999999999 947
-7.5 947
-7.450000000000001 947
-7.3999999999999995 947
-7.3500000000000005 947
-7.299999999999999 947
-7.25 947
-7.200000000000001 947
-7.1499999999999995 947
-7.1000000000000005 947
-7.049999999999999 947

In [80]:
voltage_val_oi = voltage_groups[-4.75]
voltage_val_oi

Int64Index([98488, 98489, 98490, 98491, 98492, 98493, 98494, 98495, 98496,
            98497,
            ...
            99425, 99426, 99427, 99428, 99429, 99430, 99431, 99432, 99433,
            99434],
           dtype='int64', length=947)

In [81]:
def plot_and_fit_gaussian(data):
    gaussian = fts.NormalisedGaussianFitter()
    fig, ax = plt.subplots()
    hist, bin_edges = np.histogram(data, bins='auto', density=True)
    bins = (bin_edges[:-1] + bin_edges[1:]) / 2
    ax.plot(bins, hist)
    
    params = [np.std(data), np.mean(data)]
    fit_data = gaussian.fit(bins, hist, initial_vals=params) 
    x = np.linspace(min(bins), max(bins), 1000)
    ax.plot(x, fit_data.fit_function(x), label=r'Gaussian fit - $\sigma$ = {:.3g}'.format(fit_data.get_param("sigma")))

    ax.legend()
    
    return fit_data

plot_and_fit_gaussian(df.iloc[voltage_val_oi]['current'])

In [82]:
df.iloc[voltage_val_oi]['current'].plot()

In [83]:
smoothed_df = df.groupby('voltage').filter(lambda x: len(x) == 947).groupby('voltage').tail(95).groupby('voltage').mean()
smoothed_df.plot()

In [84]:
filtered_df = df.groupby('voltage').filter(lambda x: len(x) == 947)
smoothed_df = filtered_df.groupby('voltage').mean()
smoothed_df

,current
voltage,
-9.95,29.986266
-9.90,29.665249
-9.85,29.463560
-9.80,28.706433
-9.75,28.575491
...,...
9.80,-324.420506
9.85,-298.097425
9.90,-265.180349


In [85]:
smoothed_df.size

400

In [26]:
iv_data = iv.IVData(smoothed_df.index.values, smoothed_df['current'].values, np.arange(0, smoothed_df.size))
iv_data.plot()

NameError: name 'smoothed_df' is not defined

## Compare old and new IV characteristic

In [87]:
fig, ax = plt.subplots()

# ax.plot(splopter_old.iv_data['I'])
# ax.plot(splopter_old.raw_data['I'])
ax.plot(splopter_old.iv_data['V'][:-1], splopter_old.iv_data['I'], label='2.13')
iv_data.plot(ax=ax, label='2.14')
ax.legend()

In [88]:
fig, ax = plt.subplots()

print(splopter_old.raw_data['I'].shape)

# ax.plot(splopter_old.iv_data['I'])
# ax.plot(splopter_old.raw_data['I'])
ax.plot(splopter_old.raw_data['I'], label='2.13')
ax.axvline(x=1138)

(1706,)


In [89]:
fig, ax = plt.subplots()

print(splopter_ctrl.raw_data['I'].shape)

big_spike_start = 1109
ta_point = np.ceil(2 * big_spike_start / 3)
ctrl_rawiv_index = np.arange(ta_point, big_spike_start).astype(np.int64)

# ax.plot(splopter_old.iv_data['I'])
# ax.plot(splopter_old.raw_data['I'])
ax.plot(splopter_ctrl.raw_data['I'][:big_spike_start+1], label='2.13')
ax.axvline(x=big_spike_start, color='grey')
ax.axvline(x=ta_point, color='red', linestyle='dotted')

(1999,)


In [90]:
sweep_start = 1335
print(len(splopter_ctrl.raw_data['V']) - sweep_start)
print(len(ctrl_rawiv_index))

time, raw_pot, raw_current_e, raw_current_i = splopter_ctrl.get_tdata_raw_iv(splopter_ctrl.tdata)
print(len(raw_pot))

fig, ax = plt.subplots(2)
ax[0].plot(raw_pot)
ax[0].plot(np.arange(189509,379016), raw_pot[189509:])
ax[0].axvline(189509, color='red')

ax[1].plot(raw_current_e, label='electron')
ax[1].plot(raw_current_i, label='ion')
ax[1].plot(raw_current_i + raw_current_e, label='total')
ax[1].legend()
ax[1].set_ylim(-500,150)

666
369
379016


(-500, 150)

In [91]:
fig, ax = plt.subplots(2, sharex=True)
ax[0].plot(splopter_ctrl.raw_data['I'][:big_spike_start+1], label='2.13 - control')
ax[0].axvline(x=big_spike_start, color='grey')
ax[0].axvline(x=ta_point, color='red', linestyle='dotted')

concatted_sweep_start = len(splopter_old.raw_data['I']) - len(ctrl_rawiv_index)
print(len(splopter_old.raw_data['V']))
ax[1].plot(splopter_old.raw_data['I'], label='2.13 - concatted')
ax[1].plot(np.arange(concatted_sweep_start, concatted_sweep_start+len(ctrl_rawiv_index)), splopter_old.raw_data['I'][concatted_sweep_start:])


ax[1].axvline(x=concatted_sweep_start, color='red', linestyle='dotted')
# iv_data.plot(ax=ax, label='2.14')

ax[0].legend()
ax[1].legend()

1707


In [92]:
print(len(raw_pot[189509:513:-513])) #-len(ctrl_rawiv_index):len(ctrl_rawiv_index)
print(len(splopter_ctrl.raw_data['I'][ctrl_rawiv_index]))

fig, ax = plt.subplots()
ax.plot(raw_pot[189509:-513:513], splopter_ctrl.raw_data['I'][ctrl_rawiv_index], label='2.13 - Control')
ax.plot(raw_pot[189509:-513:513], splopter_old.raw_data['I'][concatted_sweep_start:], label='2.13 - Concatted')
ax.plot(splopter_old.iv_data['V'][:-1], splopter_old.iv_data['I'], label='2.13 - Old Concatenation')

iv_data.plot(ax=ax, label='2.14')
ax.legend()

369
369


In [93]:
print(ta_point)
print(concatted_sweep_start)
ta_point / concatted_sweep_start

740.0
1337


0.5534779356768885

In [94]:
fig, ax = plt.subplots()
# ax.plot(splopter_old.iv_data['V'][:-1], splopter_old.iv_data['I'], label='2.13 - Old Concatenation')
ax.plot(splopter_old.iv_data['V'][:350], splopter_old.iv_data['I'][:350], label='2.13 - Old Concatenation')
fitter = fts.FullIVFitter()

old_d_data = fitter.fit(splopter_old.iv_data['V'][:350], splopter_old.iv_data['I'][:350])
old_d_data.plot(ax=ax)

fig, ax = plt.subplots()
# ax.plot(iv_data['V'], iv_data['I'], label='New')
ax.plot(iv_data['V'][:165], iv_data['I'][:165], label='New')
new_f_data = fitter.fit(iv_data['V'][:165], iv_data['I'][:165])
new_f_data.plot(ax=ax)


/home/jleland/coding/projects/flopter/flopter/core/fitters.py:246: RuntimeWarning: overflow encountered in exp
  return I_0 * (1 - np.exp(-V) + np.where(v <= v_f, (a * np.float_power(np.absolute(V), [0.75])), 0))
